#  Сверточные нейронные сети
Реализация стеккинга обучения моделей и метамадели на датасете из игральных карт

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data
import torch.nn.functional as F
import torchvision
from torchvision import transforms
import pandas as pd
from PIL import Image

device = "cuda"

In [3]:
torch.__version__
torch.cuda.get_device_name(0)

'NVIDIA GeForce GTX 1660 Ti with Max-Q Design'

In [8]:
def check_image(path):
    try:
        im = Image.open(path)
        return True
    except:
        return False


img_transforms = transforms.Compose([
    transforms.Resize((64,64)), 
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                    std=[0.229, 0.224, 0.225])
    ])


train_data_path = "./train/"
train_data = torchvision.datasets.ImageFolder(root=train_data_path,transform=img_transforms, is_valid_file=check_image)
val_data_path = "./valid/"
val_data = torchvision.datasets.ImageFolder(root=val_data_path,transform=img_transforms, is_valid_file=check_image)
test_data_path = "./test/"
test_data = torchvision.datasets.ImageFolder(root=test_data_path,transform=img_transforms, is_valid_file=check_image) 

batch_size=64

train_data_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size)
val_data_loader  = torch.utils.data.DataLoader(val_data, batch_size=batch_size) 
test_data_loader  = torch.utils.data.DataLoader(test_data, batch_size=batch_size) 

for batch_ndx, sample in enumerate(train_data_loader):
    print(sample[0].shape)
    print(batch_ndx)

torch.Size([64, 3, 64, 64])
0
torch.Size([64, 3, 64, 64])
1
torch.Size([64, 3, 64, 64])
2
torch.Size([64, 3, 64, 64])
3
torch.Size([64, 3, 64, 64])
4
torch.Size([64, 3, 64, 64])
5
torch.Size([64, 3, 64, 64])
6
torch.Size([64, 3, 64, 64])
7
torch.Size([64, 3, 64, 64])
8
torch.Size([64, 3, 64, 64])
9
torch.Size([64, 3, 64, 64])
10
torch.Size([64, 3, 64, 64])
11
torch.Size([64, 3, 64, 64])
12
torch.Size([64, 3, 64, 64])
13
torch.Size([64, 3, 64, 64])
14
torch.Size([64, 3, 64, 64])
15
torch.Size([64, 3, 64, 64])
16
torch.Size([64, 3, 64, 64])
17
torch.Size([64, 3, 64, 64])
18
torch.Size([64, 3, 64, 64])
19
torch.Size([64, 3, 64, 64])
20
torch.Size([64, 3, 64, 64])
21
torch.Size([64, 3, 64, 64])
22
torch.Size([64, 3, 64, 64])
23
torch.Size([64, 3, 64, 64])
24
torch.Size([64, 3, 64, 64])
25
torch.Size([64, 3, 64, 64])
26
torch.Size([64, 3, 64, 64])
27
torch.Size([64, 3, 64, 64])
28
torch.Size([64, 3, 64, 64])
29
torch.Size([64, 3, 64, 64])
30
torch.Size([64, 3, 64, 64])
31
torch.Size([64, 3,

In [14]:
file_path_test ="./cards.csv"
data = pd.read_csv(file_path_test, encoding='cp1251', )

FileNotFoundError: [Errno 2] No such file or directory: './cards.csv'

In [ ]:
len(data["labels"].unique())



53

In [11]:
class CNNNet(nn.Module):

    def __init__(self, num_classes=53):
        super().__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=11, stride=4, padding=2),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Conv2d(64, 192, kernel_size=5, padding=2),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Conv2d(192, 384, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(384, 256, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=3, stride=2),
        )
        self.avgpool = nn.AdaptiveAvgPool2d((6, 6))
        self.classifier = nn.Sequential(
            nn.Dropout(),
            nn.Linear(256 * 6 * 6, 4096),
            nn.ReLU(),
            nn.Dropout(),
            nn.Linear(4096, 4096),
            nn.ReLU(),
            nn.Linear(4096, num_classes)
        )
    
    def forward(self, x):
        x = self.features(x)
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        return x

In [12]:
cnnnet = CNNNet()

----

In [ ]:
cnnnet(torch.rand(1, 3,224, 224, requires_grad=True))[0].shape

torch.Size([53])

In [6]:
def train(model, optimizer, loss_fn, train_loader, val_loader, epochs=20, device="cuda:0"):
    acc = 0
    for epoch in range(1, epochs+1):
        training_loss = 0.0
        valid_loss = 0.0
        model.train()

        for batch in train_loader:
            optimizer.zero_grad()
            inputs, targets = batch
            inputs = inputs.to(device)
            targets = targets.to(device)
            output = model(inputs)
            loss = loss_fn(output, targets)
            loss.backward()
            optimizer.step()
            training_loss += loss.data.item() * inputs.size(0)
        training_loss /= len(train_loader.dataset)
        
        model.eval()
        
        num_correct = 0 
        num_examples = 0
        for batch in val_loader:
            inputs, targets = batch
            inputs = inputs.to(device)
            output = model(inputs)
            targets = targets.to(device)
            loss = loss_fn(output,targets) 
            valid_loss += loss.data.item() * inputs.size(0)
            correct = torch.eq(torch.max(F.softmax(output, dim=1), dim=1)[1],
                               targets)
            num_correct += torch.sum(correct).item()
            num_examples += correct.shape[0]
        valid_loss /= len(val_loader.dataset)
        if (acc< num_correct / num_examples):
                acc = num_correct / num_examples
                
        if (19< epoch):
                acc = num_correct / num_examples
                torch.save(model,f'model{epoch}_{acc}.pt')        
        print('Epoch: {}, Training Loss: {:.2f}, Validation Loss: {:.2f}, accuracy = {:.2f}'.format(epoch, training_loss,
        valid_loss, num_correct / num_examples))

In [7]:
def check_image(path):
    try:
        im = Image.open(path)
        return True
    except:
        return False

img_transforms = transforms.Compose([
    transforms.Resize((224,224)),    
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
    ])
train_data_path = "./train/"
train_data = torchvision.datasets.ImageFolder(root=train_data_path,transform=img_transforms, is_valid_file=check_image)
val_data_path = "./valid/"
val_data = torchvision.datasets.ImageFolder(root=val_data_path,transform=img_transforms, is_valid_file=check_image)
batch_size=32
train_data_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size,shuffle=True)
val_data_loader  = torch.utils.data.DataLoader(val_data, batch_size=batch_size, shuffle=True)
device = torch.device("cuda") 


In [8]:
torch.cuda.is_available()

True

In [15]:
cnnnet.to(device)
optimizer = optim.Adam(cnnnet.parameters(), lr=0.001)

NameError: name 'device' is not defined

In [9]:
# Загрузка предварительно обученной модели MobileNet
model = torch.hub.load('pytorch/vision:v0.10.0', 'mobilenet_v2', pretrained=True)

# Перемещение модели на устройство (GPU или CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Пример загрузки изображения и его преобразования в тензор
input_image = Image.open("./train/ace of clubs/001.jpg")
preprocess = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Применение преобразований к изображению
input_tensor = preprocess(input_image)

# Добавление размерности батча (batch dimension) для ожидаемого входа модели
input_tensor = input_tensor.unsqueeze(0)

# Передача тензора на устройство (GPU или CPU)
input_tensor = input_tensor.to(device)

# Изменение последнего слоя классификатора модели для адаптации к вашим данным
num_classes = 53
model.classifier[1] = nn.Linear(1280, num_classes)

# Определение функции потерь и оптимизатора
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)




Using cache found in C:\Users\anon/.cache\torch\hub\pytorch_vision_v0.10.0
C:\Users\anon\AppData\Roaming\Python\Python310\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\anon\AppData\Roaming\Python\Python310\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=MobileNet_V2_Weights.IMAGENET1K_V1`. You can also use `weights=MobileNet_V2_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [91]:
import torch
import torchvision
from torch import nn, optim
from torchvision import transforms, datasets
from PIL import Image
from torch.nn import functional as F
from torch.quantization import QuantStub, DeQuantStub


# Определение функции обучения
# Определение функции обучения
def train(model, optimizer, loss_fn, train_loader, val_loader, epochs=20, device="cuda:0"):
    acc = 0
    for epoch in range(1, epochs+1):
        training_loss = 0.0
        valid_loss = 0.0
        model.train()

        for batch in train_loader:
            optimizer.zero_grad()
            inputs, targets = batch
            inputs = inputs.to(device)
            targets = targets.to(device)
            # Перемещаем модель на устройство
            model.to(device)
            output = model(inputs)
            loss = loss_fn(output, targets)
            loss.backward()
            optimizer.step()
            training_loss += loss.item() * inputs.size(0)
        training_loss /= len(train_loader.dataset)
        
        model.eval()
        
        num_correct = 0 
        num_examples = 0
        for batch in val_loader:
            inputs, targets = batch
            inputs = inputs.to(device)
            # Перемещаем модель на устройство
            model.to(device)
            output = model(inputs)
            targets = targets.to(device)
            loss = loss_fn(output, targets) 
            valid_loss += loss.item() * inputs.size(0)
            correct = torch.eq(torch.max(F.softmax(output, dim=1), dim=1)[1], targets)
            num_correct += torch.sum(correct).item()
            num_examples += correct.shape[0]
        valid_loss /= len(val_loader.dataset)
        
        if (acc< num_correct / num_examples):
            acc = num_correct / num_examples
                
        if (19< epoch):
            acc = num_correct / num_examples
            torch.save(model, f'model{epoch}_{acc}.pt')        
        print('Epoch: {}, Training Loss: {:.2f}, Validation Loss: {:.2f}, accuracy = {:.2f}'.format(epoch, training_loss,
        valid_loss, num_correct / num_examples))

batch_size = 16
train_data_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, shuffle=True)
val_data_loader = torch.utils.data.DataLoader(val_data, batch_size=batch_size, shuffle=True)
# Функция проверки изображения
def check_image(path):
    try:
        im = Image.open(path)
        return True
    except:
        return False

# Трансформации для изображений
img_transforms = transforms.Compose([
    transforms.Resize((224,224)),    
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Пути к данным
train_data_path = "./train/"
val_data_path = "./valid/"

# Создание датасетов
train_data = torchvision.datasets.ImageFolder(root=train_data_path, transform=img_transforms, is_valid_file=check_image)
val_data = torchvision.datasets.ImageFolder(root=val_data_path, transform=img_transforms, is_valid_file=check_image)

# DataLoader для обучения и валидации
batch_size = 10
train_data_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, shuffle=True)
val_data_loader = torch.utils.data.DataLoader(val_data, batch_size=batch_size, shuffle=True)

# Определение устройства (GPU или CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Загрузка предварительно обученной модели MobileNet
model = torch.hub.load('pytorch/vision:v0.10.0', 'mobilenet_v2', pretrained=True)

# Перемещение модели на устройство (GPU или CPU)
model.to(device)

# Изменение последнего слоя классификатора модели для адаптации к вашим данным
num_classes = 53
model.classifier[1] = nn.Linear(1280, num_classes)

# Определение функции потерь и оптимизатора
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)




Using cache found in C:\Users\anon/.cache\torch\hub\pytorch_vision_v0.10.0


In [92]:
train(model, optimizer, criterion, train_data_loader, val_data_loader, epochs=2, device=device)

KeyboardInterrupt: 

In [12]:
model.eval()
#model.half()
x = torch.rand(1, 3,224, 224, requires_grad=True).to(device)
torch_out = model(x)

In [96]:
# Сохранение модели в формате ONNX
torch.onnx.export(model,                   # Модель
                  torch.randn(1, 3, 224, 224).to(device), # Пример входных данных
                  "mobile_FP16.onnx",            # Путь для сохранения
                  export_params=True,       # Сохранять ли параметры модели
                  opset_version=11,         # Версия ONNX
                  do_constant_folding=True, # Оптимизировать ли константные узлы
                  input_names=['input'],    # Имена входных узлов
                  output_names=['output'],  # Имена выходных узлов
                  dynamic_axes={'input': {0: 'batch_size'}, # Динамические оси
                                'output': {0: 'batch_size'}})

In [17]:
import onnx
import onnxruntime
import numpy as np

In [24]:


def predict_image(model_path, image_path, img_transforms, device):
    # Загрузка ONNX модели
    session = onnxruntime.InferenceSession(model_path)

    # Открытие изображения
    image = Image.open(image_path)

    # Применение преобразований к изображению
    image = img_transforms(image).unsqueeze(0)

    # Получение входного имени и выходного имени из модели
    input_name = session.get_inputs()[0].name
    output_name = session.get_outputs()[0].name

    # Выполнение предсказания
    input_data = image.cpu().numpy() if device == torch.device("cuda") else image.numpy()

    outputs = session.run([output_name], {input_name: input_data})

    # Получение индекса предсказанного класса
    predicted_class = np.argmax(outputs[0], axis=1)

    return predicted_class.item()




In [22]:
class_names = train_data.classes

In [22]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import os
import numpy as np
import torch
import torchvision.transforms as transforms
from PIL import Image
import onnxruntime

# Функция для предсказания класса изображения
def predict_image(model_path, image_path, img_transforms, device):
    # Загрузка ONNX модели
    session = onnxruntime.InferenceSession(model_path)

    # Открытие изображения
    image = Image.open(image_path)

    # Применение преобразований к изображению
    image = img_transforms(image).unsqueeze(0)

    # Получение входного имени и выходного имени из модели
    input_name = session.get_inputs()[0].name
    output_name = session.get_outputs()[0].name

    # Выполнение предсказания
    input_data = image.cpu().numpy() if device == torch.device("cuda") else image.numpy()
    outputs = session.run([output_name], {input_name: input_data})

    # Получение индекса предсказанного класса
    predicted_class = np.argmax(outputs[0], axis=1)

    return predicted_class.item()

# Путь к вашей модели ONNX
model_path = "mobile.onnx"

# Путь к вашему тестовому набору данных
test_data_path = "./test/"


# Преобразования изображений
img_transforms = transforms.Compose([
    transforms.Resize((224, 224)),    
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

In [97]:
predicted_class = predict_image("./mobile_FP16.onnx", "./test/ace of clubs/5.jpg", img_transforms, device)
print("Предсказанный класс:", predicted_class)

class_names = train_data.classes
print("Имя класса :",class_names[predicted_class])

Предсказанный класс: 0
Имя класса : ace of clubs


In [28]:
for _ in range(1000):
    predicted_class = predict_image("./mobile2.onnx", "./test/ace of clubs/5.jpg", img_transforms, device)




получилось 28.4 секунды на модели до оптимизации

Accuracy: 0.8981132075471698
Precision: 0.92047020065888
Recall: 0.8981132075471698
F1 Score: 0.8961433744452614 это метрики mobilenet

---------------------------------------


In [ ]:
import onnxruntime

providers = onnxruntime.get_available_providers()
print(providers)


import onnxruntime as ort

ort.get_device()

In [34]:
import onnxruntime as rt

sess_options = rt.SessionOptions()

sess_options.optimized_model_filepath = "mobile2.onnx" 
 
session = rt.InferenceSession("./mobile.onnx", sess_options,providers=["CUDAExecutionProvider"])

Таких больших моделей у меня нет поэтому я для видимости результата помещу функцию предсказания в цикл на 1000 раз и будущие оптимизированные модели будут также следовать этому циклу и тогда можно будет узнать эффективность оптимизации в режиме так называемого стресс теста 

In [38]:
input_shape = (1, 3, 224, 224)
input_data = torch.randn(*input_shape).to(device)
for _ in range(1000):
    prediction = session.run(None, {"input": input_data.cpu().numpy()})

9.6 секунды на модели до оптимизации

In [39]:
import onnxruntime as rt

sess_options = rt.SessionOptions()
sess_options.graph_optimization_level = rt.GraphOptimizationLevel.ORT_ENABLE_ALL

# Загрузка модели и создание сессии с ONNX Runtime
session = rt.InferenceSession("mobile2.onnx", sess_options, providers=["CUDAExecutionProvider"])

# Подготовка входных данных для модели
input_shape = (1, 3, 224, 224)
input_data = torch.randn(*input_shape).to(device)


onnx_model = session.get_model()

# Сохранение модели в формате ONNX
onnx.save(onnx_model, "optimized_mobile2.onnx")

In [41]:
input_shape = (1, 3, 224, 224)
input_data = torch.randn(*input_shape).to(device)
for _ in range(1000):
    prediction = session.run(None, {"input": input_data.cpu().numpy()})

9.3 секунды на модели после оптимизации

In [6]:
import torch
import torchvision
from torch import nn, optim
from torchvision import transforms, datasets
from PIL import Image
from torch.nn import functional as F
from torch.quantization import QuantStub, DeQuantStub
from torch.cuda.amp import GradScaler, autocast

# Определение функции обучения
def train(model, optimizer, loss_fn, train_loader, val_loader, epochs=20, device="cuda:0"):
    acc = 0
    scaler = GradScaler()
    
    for epoch in range(1, epochs+1):
        training_loss = 0.0
        valid_loss = 0.0
        model.train()

        for batch in train_loader:
            optimizer.zero_grad()
            inputs, targets = batch
            inputs = inputs.to(device)
            targets = targets.to(device)
            
            with autocast():
                output = model(inputs)
                loss = loss_fn(output, targets)
            
            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()

            training_loss += loss.item() * inputs.size(0)
            
        training_loss /= len(train_loader.dataset)
        
        model.eval()
        
        num_correct = 0 
        num_examples = 0
        
        for batch in val_loader:
            inputs, targets = batch
            inputs = inputs.to(device)
            targets = targets.to(device)
            
            with torch.no_grad():
                with autocast():
                    output = model(inputs)
                    loss = loss_fn(output, targets)
            
            valid_loss += loss.item() * inputs.size(0)
            
            correct = torch.eq(torch.max(F.softmax(output, dim=1), dim=1)[1], targets)
            num_correct += torch.sum(correct).item()
            num_examples += correct.shape[0]
        
        valid_loss /= len(val_loader.dataset)
        
        if acc < num_correct / num_examples:
            acc = num_correct / num_examples
                
        if 19 < epoch:
            acc = num_correct / num_examples
            torch.save(model, f'model{epoch}_{acc}.pt')        
            
        print('Epoch: {}, Training Loss: {:.2f}, Validation Loss: {:.2f}, accuracy = {:.2f}'.format(epoch, training_loss, valid_loss, num_correct / num_examples))

# Функция проверки изображения
def check_image(path):
    try:
        im = Image.open(path)
        return True
    except:
        return False

# Трансформации для изображений
img_transforms = transforms.Compose([
    transforms.Resize((224,224)),    
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Пути к данным
train_data_path = "./train/"
val_data_path = "./valid/"

# Создание датасетов
train_data = datasets.ImageFolder(root=train_data_path, transform=img_transforms, is_valid_file=check_image)
val_data = datasets.ImageFolder(root=val_data_path, transform=img_transforms, is_valid_file=check_image)

# DataLoader для обучения и валидации
batch_size = 16  # Изменено для более эффективного использования FP16
train_data_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, shuffle=True)
val_data_loader = torch.utils.data.DataLoader(val_data, batch_size=batch_size, shuffle=True)

# Определение устройства (GPU или CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Загрузка предварительно обученной модели MobileNet
model = torch.hub.load('pytorch/vision:v0.10.0', 'mobilenet_v2', pretrained=True)

# Перемещение модели на устройство (GPU или CPU)
model.to(device)

# Изменение последнего слоя классификатора модели для адаптации к вашим данным
num_classes = 53
model.classifier[1] = nn.Linear(1280, num_classes)

# Определение функции потерь и оптимизатора
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)







Using cache found in C:\Users\anon/.cache\torch\hub\pytorch_vision_v0.10.0
C:\Users\anon\AppData\Roaming\Python\Python310\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\anon\AppData\Roaming\Python\Python310\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=MobileNet_V2_Weights.IMAGENET1K_V1`. You can also use `weights=MobileNet_V2_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [95]:
# Обучение моделиmodel.to(device)
model.to(device)
train(model, optimizer, criterion, train_data_loader, val_data_loader, epochs=20, device=device)


Epoch: 1, Training Loss: 1.88, Validation Loss: 0.62, accuracy = 0.81
Epoch: 2, Training Loss: 0.74, Validation Loss: 0.31, accuracy = 0.89
Epoch: 3, Training Loss: 0.50, Validation Loss: 0.33, accuracy = 0.89
Epoch: 4, Training Loss: 0.39, Validation Loss: 0.14, accuracy = 0.96
Epoch: 5, Training Loss: 0.33, Validation Loss: 0.06, accuracy = 0.98
Epoch: 6, Training Loss: 0.29, Validation Loss: 0.09, accuracy = 0.98
Epoch: 7, Training Loss: 0.27, Validation Loss: 0.06, accuracy = 0.98
Epoch: 8, Training Loss: 0.25, Validation Loss: 0.12, accuracy = 0.97
Epoch: 9, Training Loss: 0.23, Validation Loss: 0.11, accuracy = 0.97
Epoch: 10, Training Loss: 0.22, Validation Loss: 0.15, accuracy = 0.96
Epoch: 11, Training Loss: 0.20, Validation Loss: 0.07, accuracy = 0.98
Epoch: 12, Training Loss: 0.20, Validation Loss: 0.09, accuracy = 0.97
Epoch: 13, Training Loss: 0.16, Validation Loss: 0.16, accuracy = 0.96
Epoch: 14, Training Loss: 0.14, Validation Loss: 0.10, accuracy = 0.97
Epoch: 15, Trai

метрики при обучении стали гораздо лучше 

In [84]:
torch.cuda.is_available()

True

In [7]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import os
import numpy as np
import torch
import torchvision.transforms as transforms
from PIL import Image
import onnxruntime

# Функция для предсказания класса изображения
def predict_image(model_path, image_path, img_transforms, device):
    # Загрузка ONNX модели
    session = onnxruntime.InferenceSession(model_path)

    # Открытие изображения
    image = Image.open(image_path)

    # Применение преобразований к изображению
    image = img_transforms(image).unsqueeze(0)

    # Получение входного имени и выходного имени из модели
    input_name = session.get_inputs()[0].name
    output_name = session.get_outputs()[0].name

    # Выполнение предсказания
    input_data = image.cpu().numpy() if device == torch.device("cuda") else image.numpy()
    outputs = session.run([output_name], {input_name: input_data})

    # Получение индекса предсказанного класса
    predicted_class = np.argmax(outputs[0], axis=1)

    return predicted_class.item()
train_data_path = "./train/"
# Создание датасетов
train_data = datasets.ImageFolder(root=train_data_path, transform=img_transforms, is_valid_file=check_image)
class_names = train_data.classes
# Путь к вашей модели ONNX
model_path = "mobile_FP16.onnx"

# Путь к вашему тестовому набору данных
test_data_path = "./test/"


# Преобразования изображений
img_transforms = transforms.Compose([
    transforms.Resize((224, 224)),    
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])


# Инициализация списка для хранения предсказанных классов и истинных меток
predicted_classes = []
true_labels = []

# Пройти по всем изображениям в тестовом наборе данных
for root, dirs, files in os.walk(test_data_path):
    for file in files:
        if file.endswith(".jpg"):  # Проверка, что файл - изображение
            image_path = os.path.join(root, file)
            true_label = os.path.basename(root)  # Извлечение истинного класса из пути
            predicted_class = predict_image(model_path, image_path, img_transforms, device)
            predicted_classes.append(predicted_class)
            true_labels.append(true_label)

untocken_classes = []
for i in predicted_classes:
    untocken_classes.append(class_names[i])

class_names

# Вычисление метрик
accuracy = accuracy_score(true_labels, untocken_classes)
precision = precision_score(true_labels, untocken_classes, average='weighted')
recall = recall_score(true_labels, untocken_classes, average='weighted')
f1 = f1_score(true_labels, untocken_classes, average='weighted')

# Вывод метрик
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)

Accuracy: 0.9660377358490566
Precision: 0.9725965858041332
Recall: 0.9660377358490566
F1 Score: 0.9654564513055078


Accuracy: 0.9660377358490566
Precision: 0.9725965858041332
Recall: 0.9660377358490566
F1 Score: 0.9654564513055078 это метрики mobilenet в режиме FP16 c использованием например GradScaler

Accuracy: 0.8981132075471698
Precision: 0.92047020065888
Recall: 0.8981132075471698
F1 Score: 0.8961433744452614 это метрики mobilenet

как видно это значительно помогло улучшить метрики моели а также  заметил что метоу понаобилось бы меньше эпох дл достижени этого результата